# App Overview

In [ ]:
# 🛠 Imports and logging setup
import logging
import datetime
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import datetime
import json
from yfinance import Ticker
from typing import List
from pandas import DataFrame
from datetime import datetime, timezone
from typing import List
from yfinance import Ticker

In [10]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def log(msg, emoji="ℹ️"):
    logging.info(f"{emoji} {msg}")

In [ ]:
# 🌍 OBSOLETE: Define sectors & scrape tickers (placeholder: extend over time)
today = datetime.date.today().strftime("%Y-%m-%d")
log(f"Fetching stock lists for {today}", "📅")

# For demo: hardcode sample tickers (can extend by scraping)
tech_tickers = ["AAPL", "MSFT", "NVDA", "GOOGL", "AMZN"]
renewable_tickers = ["NEE", "ENPH", "PLUG", "FSLR", "SEDG"]
manufacturing_tickers = ["GE", "CAT", "DE", "BA", "HON"]

pruned_tickers = tech_tickers + renewable_tickers + manufacturing_tickers
log(f"Collected {len(pruned_tickers)} tickers (sample universe)", "📊")

universe_df = pd.DataFrame({
    "Ticker": pruned_tickers,
    "Sector": ["Tech"]*len(tech_tickers) + ["Renewable"]*len(renewable_tickers) + ["Manufacturing"]*len(manufacturing_tickers)
})
universe_df


In [ ]:
# 🌍 Fetch top 50 live high dividend tickers (via screener + yield filter)
today = datetime.date.today().strftime("%Y-%m-%d")
log(f"Fetching live high dividend stocks for {today}", "📅")

import requests, yfinance as yf

gross_ticker_count = 100
scrId = "most_actives"  # wide enough universe
api_url = (
    f"https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved?"
    f"formatted=true&scrIds={scrId}&count={gross_ticker_count}&start=0"
)

resp = requests.get(api_url, headers={"User-Agent": "Mozilla/5.0"})
data = resp.json()
quotes = (
    data.get("finance", {})
        .get("result", [{}])[0]
        .get("quotes", [])
)

log(f"Collected {len(quotes)} tickers from Yahoo screener '{scrId}'", "📊")

# ✅ Filter tickers by dividend yield
tickers = []
for q in quotes:
    t = q.get("symbol")
    if not t:
        continue
    try:
        stock_info_json = yf.Ticker(t).info
        dy = stock_info_json.get("dividendYield")
        if dy and dy > 0.03:  # only >3% dividend yield
            tickers.append(t)
            log(f"{t} passes dividend filter ({dy*100:.2f}%)", "✅")
    except Exception as e:
        log(f"Skip {t}: {e}", "⚠️")

pruned_tickers = tickers[:50]  # 🔑 <-- your original flow preserved
log(f"Final selection: {len(pruned_tickers)} high dividend tickers", "📊")

universe_df = pd.DataFrame({
    "Ticker": pruned_tickers,
    "Sector": "High Dividend"
})
universe_df


In [ ]:
# 🌍 Fetch tickers from Yahoo screener (generic)
today = datetime.date.today().strftime("%Y-%m-%d")
log(f"Fetching screener results for {today}", "📅")

# You can change scrId here: "day_gainers", "most_actives", "day_losers", etc.
scrId = "day_gainers"

api_url = (
    f"https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved?"
    f"formatted=true&scrIds={scrId}&count=50&start=0"
)

resp = requests.get(api_url, headers={"User-Agent": "Mozilla/5.0"})
log(f"HTTP status code: {resp.status_code}", "🌐")

data = resp.json()
quotes = (
    data.get("finance", {})
        .get("result", [{}])[0]
        .get("quotes", [])
)

log(f"Found {len(quotes)} quotes under screener '{scrId}'", "📑")

tickers = [
    {"Ticker": q.get("symbol"), "Name": q.get("longName", q.get("shortName"))}
    for q in quotes if q.get("symbol")
]

universe_df = pd.DataFrame(tickers)
universe_df["Sector"] = scrId
log(f"Collected {len(universe_df)} tickers from Yahoo screener '{scrId}'", "📊")

universe_df


In [ ]:
# 💰 Download dividend & yield info from Yahoo Finance
log("Downloading dividend info from Yahoo Finance...", "⬇️")
tickers = pruned_tickers[:50]
data = []
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        stock_info_json = stock.info
        dividends = stock.dividends

        dividend_yield = stock_info_json.get("dividendYield") or 0
        five_year_yield = stock_info_json.get("fiveYearAvgDividendYield") or 0
        currency = stock_info_json.get("currency") or "USD"
        annual_div = dividends[-252:].sum() if not dividends.empty else 0

        data.append({
            "Ticker": ticker,
            "Name": stock_info_json.get("longName"),
            "Sector": universe_df.loc[universe_df["Ticker"]==ticker, "Sector"].values[0],
            "Currency": currency,
            "Dividend Yield %": round(dividend_yield*100, 2),
            "5Y Avg Yield %": round(five_year_yield, 2),
            "Annual Dividend": round(annual_div, 2)
        })
        log(f"Processed {ticker}", "✅")
    except Exception as e:
        log(f"Failed for {ticker}: {e}", "❌")


In [ ]:
data


In [146]:
def stocks_to_pd(stocks: List[Ticker])->DataFrame:
    rows = []
    for stock in stocks:
        #stock = yf.Ticker(t)
        stock_info_json = stock.info or {}
        rows.append({
            "🔍": stock_info_json.get("longName"),
            "displayName": stock_info_json.get("displayName"),
            "symbol": stock_info_json.get("symbol"),
            "website": stock_info_json.get("website"),
            "exchange": stock_info_json.get("exchange"),
            "country": stock_info_json.get("country"),
            "fiveYearAvgDividendYield": stock_info_json.get("fiveYearAvgDividendYield"),
            "dividendRate": stock_info_json.get("dividendRate"),
            "dividendYield": stock_info_json.get("dividendYield"),
            "currentDividendYield": calc_current_div_yield(stock),
            "lastDividendDate": to_date(stock_info_json.get("lastDividendDate")),
            "dividendDate": to_date(stock_info_json.get("dividendDate")),
            "exDividendDate": to_date(stock_info_json.get("exDividendDate")),
            "lastDividendValue": stock_info_json.get("lastDividendValue"),
            "currentPrice": stock_info_json.get("currentPrice"),
            "quoteType": stock_info_json.get("quoteType"),
            "industry": stock_info_json.get("industry"),
            "sharesOutstanding": stock_info_json.get("sharesOutstanding"),
            "currency": stock_info_json.get("currency"),
            "ask": stock_info_json.get("ask"),
            "askSize": stock_info_json.get("askSize"),
            "previousClose": stock_info_json.get("previousClose"),
            "market": stock_info_json.get("market"),
            "marketCap": stock_info_json.get("marketCap"),
            "fiftyDayAverage": stock_info_json.get("fiftyDayAverage"),
            "fiftyTwoWeekHigh": stock_info_json.get("fiftyTwoWeekHigh"),
            "fiftyTwoWeekLow": stock_info_json.get("fiftyTwoWeekLow"),
            "boardRisk": stock_info_json.get("boardRisk"),
        })

    return pd.DataFrame(rows)

In [ ]:
def from_unix_datetime(ts: int):
    """Convert UNIX timestamp (seconds) → UTC timezone-aware datetime."""
    if ts is None:
        return None
    try:
        return datetime.fromtimestamp(int(ts), tz=timezone.utc)
    except Exception:
        return None
    
def to_date(unix_ts: int):
    """Convert UNIX timestamp (seconds) → UTC timezone-aware datetime."""
    if unix_ts is None:
        return None
    try:
        return datetime.fromtimestamp(int(unix_ts), tz=timezone.utc).date()
    except Exception:
        return None    



In [ ]:
def print_stock_info(stock:Ticker):
    stock_info_json = stock.info or {}
    #print(pretty_print_json(stock_info_json))
    print("=======================")
    print(f"\n🔍 {t} — {stock_info_json.get('longName')}")
    #print("dividendYield raw:", stock_info.get("dividendYield"))
    print("displayName:", stock_info_json.get("displayName"))
    print("symbol:", stock_info_json.get("symbol"))
    print("website:", stock_info_json.get("website"))
    print("exchange:", stock_info_json.get("exchange"))
    print("country:", stock_info_json.get("country"))
    print("fiveYearAvgDividendYield:", stock_info_json.get("fiveYearAvgDividendYield"))
    print("dividendRate:", stock_info_json.get("dividendRate")) # Expected
    print("dividendYield:", stock_info_json.get("dividendYield")) # Based on price between previous
    print(f"currentDividendYield:{calc_current_div_yield(stock):.2f}") # Forward Dividend Rate ÷ Current Share Price × 100 %.
    print("lastDividendDate:", to_date(stock_info_json.get("lastDividendDate")))
    print("dividendDate:", to_date(stock_info_json.get("dividendDate")))
    print("exDividendDate:", to_date(stock_info_json.get("exDividendDate")))
    print("lastDividendValue:", stock_info_json.get("lastDividendValue"))
    print("currentPrice:", stock_info_json.get("currentPrice"))
    print("quoteType:", stock_info_json.get("quoteType"))
    print("industry:", stock_info_json.get("industry"))
    #print("sharesOutstanding:", stock_info.get("sharesOutstanding"))
    #print("currency:", stock_info.get("currency"))
    #print("ask:", stock_info.get("ask"))
    #print("askSize:", stock_info.get("askSize"))
    #print("previousClose:", stock_info.get("previousClose"))
    #print("market:", stock_info.get("market"))
    #print("marketCap:", stock_info.get("marketCap"))
    #print("fiftyDayAverage:", stock_info.get("fiftyDayAverage"))
    #print("fiftyTwoWeekHigh:", stock_info.get("fiftyTwoWeekHigh"))
    #print("fiftyTwoWeekLow:", stock_info.get("fiftyTwoWeekLow"))
    #print("boardRisk:", stock_info.get("boardRisk"))

In [ ]:
def calc_current_div_yield(stock: Ticker)->float:
    stock_info = stock.info or {}
    return round((stock_info.get("dividendRate")/stock_info.get("currentPrice"))*100,2)

In [56]:
def to_pct(raw):
    if raw is None:
        pct = None
    else:
        pct = round(raw * 100, 2) if raw <= 1 else round(raw, 2)
    return pct

In [ ]:
# 💰 Download dividend + yield + totals (non-breaking, robust)
from tqdm import tqdm
log("Downloading dividend, yields, and sharesOutstanding…", "⬇️")

data = []
stocks: List[Ticker] = []
try:
    for ticker_symbol in tqdm(pruned_tickers, desc="Fetching stocks"):
        stock = yf.Ticker(ticker_symbol)
        stocks.append(stock)
        #log(f"Downloaded {ticker_symbol}", "✅")
        #print_stock_info(stock)
except Exception as e:
    log(f"Failed to download stock information for ticker symbol '{ticker_symbol}': {e}", "❌")

stocks_pd = stocks_to_pd(stocks)
stocks_pd.sort_values("currentDividendYield", ascending=False, inplace=True)
stocks_pd

    


2025-09-18 12:19:05,565 - INFO - ⬇️ Downloading dividend, yields, and sharesOutstanding…
Fetching stocks:   0%|          | 0/37 [00:00<?, ?it/s]2025-09-18 12:19:05,567 - INFO - ✅ Downloaded BBD
2025-09-18 12:19:05,567 - INFO - ✅ Downloaded F
2025-09-18 12:19:05,574 - INFO - ✅ Downloaded BTG
2025-09-18 12:19:05,575 - INFO - ✅ Downloaded PFE
2025-09-18 12:19:05,576 - INFO - ✅ Downloaded ABEV
2025-09-18 12:19:05,577 - INFO - ✅ Downloaded NOK
2025-09-18 12:19:05,578 - INFO - ✅ Downloaded VALE
2025-09-18 12:19:05,579 - INFO - ✅ Downloaded AAPL
2025-09-18 12:19:05,580 - INFO - ✅ Downloaded INFY
2025-09-18 12:19:05,582 - INFO - ✅ Downloaded BAC
2025-09-18 12:19:05,584 - INFO - ✅ Downloaded GOOGL
2025-09-18 12:19:05,585 - INFO - ✅ Downloaded ITUB
2025-09-18 12:19:05,587 - INFO - ✅ Downloaded KEY
2025-09-18 12:19:05,589 - INFO - ✅ Downloaded BABA
2025-09-18 12:19:05,592 - INFO - ✅ Downloaded HBAN
2025-09-18 12:19:05,594 - INFO - ✅ Downloaded B
2025-09-18 12:19:05,596 - INFO - ✅ Downloaded JD
20

,🔍,displayName,symbol,website,exchange,country,fiveYearAvgDividendYield,dividendRate,dividendYield,currentDividendYield,...,currency,ask,askSize,previousClose,market,marketCap,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,boardRisk
33,AGNC Investment Corp.,AGNC Inv,AGNC,https://agnc.com,NMS,United States,13.11,1.44,14.09,14.10,...,USD,10.26,35,10.130,us_market,10754422784,9.6972,10.85,7.850000,4.0
29,Petróleo Brasileiro S.A. - Petrobras,None,PBR,https://petrobras.com.br,NYQ,Brazil,21.69,1.82,14.10,14.09,...,USD,12.90,292,12.940,us_market,80121782272,12.4602,15.34,11.030000,NaN
6,Vale S.A.,Vale,VALE,https://vale.com,NYQ,Brazil,9.08,1.18,10.86,10.85,...,USD,11.00,262,10.910,us_market,46444314624,10.1106,12.05,8.060000,NaN
31,Stellantis N.V.,Stellantis,STLA,https://www.stellantis.com,NYQ,Netherlands,NaN,0.77,7.95,7.95,...,USD,9.85,256,9.720,us_market,27962849280,9.5012,16.29,8.390000,4.0
4,Ambev S.A.,Ambev,ABEV,https://www.ambev.com.br,NYQ,Brazil,4.40,0.18,7.45,7.63,...,USD,2.37,1420,2.360,us_market,37264105472,2.2726,2.64,1.760000,NaN
3,Pfizer Inc.,Pfizer,PFE,https://www.pfizer.com,NYQ,United States,4.69,1.72,7.15,7.15,...,USD,24.21,52,23.900,us_market,136737488896,24.7184,30.43,20.920000,6.0
30,Amcor plc,Amcor,AMCR,https://www.amcor.com,NYQ,Switzerland,4.56,0.51,6.16,6.16,...,USD,8.34,573,8.390,us_market,19113220096,9.0462,11.48,8.160000,7.0
1,Ford Motor Company,None,F,https://www.ford.com,NYQ,United States,5.76,0.60,5.15,5.15,...,USD,0.00,129,11.610,us_market,46405169152,11.4636,11.99,8.440000,10.0
0,Banco Bradesco S.A.,Banco Bradesco,BBD,https://banco.bradesco,NYQ,Brazil,4.49,0.17,5.19,5.14,...,USD,0.00,1792,3.210,us_market,31410081792,2.9574,3.34,1.840000,NaN
32,Kenvue Inc.,Kenvue,KVUE,https://www.kenvue.com,NYQ,United States,NaN,0.83,4.53,4.53,...,USD,0.00,26,18.130,us_market,35195744256,20.9854,25.17,17.145000,4.0


In [ ]:
data

In [145]:
# 📊 Create overview DataFrame
df = pd.DataFrame(data)
df

""


In [ ]:
# 🔄 Pivot examples for analysis
pivot_sector = df.pivot_table(
    values="Annual Net (M)",
    index="Sector",
    aggfunc="sum"
).sort_values("Annual Net (M)", ascending=False)

pivot_sector


In [ ]:
# TODO: Throwaway code
vym = yf.Ticker("VYM")
vym_holdings = vym.funds_holdings
top50 = vym_holdings.head(50)

universe_df = pd.DataFrame({
    "Ticker": top50["symbol"],
    "Name": top50["holdingName"],
    "Sector": "High Dividend"
})
log(f"Collected {len(universe_df)} tickers from VYM ETF", "📊")
universe_df


In [ ]:
# TODO: Throwaway code
test_url = "https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved?scrIds=day_gainers&count=5"
print(requests.get(test_url, headers={"User-Agent": "Mozilla/5.0"}).json())

In [ ]:
# TODO: Possibly throwaway code
# 📈 Add yield % if present
if "dividendYield" in df.columns:
    df["Dividend Yield %"] = (df["dividendYield"] * 100).round(2)

if "fiveYearAvgDividendYield" in df.columns:
    df["5Y Avg Yield %"] = (df["fiveYearAvgDividendYield"] * 100).round(2)

# 🧮 Per-share dividend
if "dividendRate" in df.columns:
    df["Annual Dividend (per share)"] = df["dividendRate"].round(2)

# 🏢 Company-wide totals (USD M)
if "dividendRate" in df.columns and "sharesOutstanding" in df.columns:
    df["Annual Gross (USD M)"] = (
        df["dividendRate"] * df["sharesOutstanding"] / 1_000_000
    ).round(2)
    df["Annual Net (USD M)"] = (df["Annual Gross (USD M)"] * 0.7).round(2)

log("Final dividend overview with explicit units ready", "📈")
df


# 🐞 Troubleshooting

In [71]:
def pretty_print_json(data: dict) -> str:
    """Return JSON string with indentation and sorted keys."""
    return json.dumps(data, indent=3, sort_keys=True, default=str)

from IPython.display import JSON

def pretty_print_json_new(data: dict):
    """Display JSON with indentation in Jupyter."""
    json_str = json.dumps(data, indent=2, sort_keys=True, default=str)
    display(JSON(json.loads(json_str)))  # still collapsible & colored


In [138]:
# 🐞 Troubleshoot WMT, GOOG, AG
test_tickers = ["WMT", "GOOG", "AG", "BBD", "NOK", "NDA-DK.CO", "NDA-FI.HE"]

stocks: List[Ticker] = []
for t in test_tickers:
    stock = yf.Ticker(t)
    stocks.append(stock)
    #print_stock_info(stock)
stocks_pd = stocks_to_pd(stocks)
stocks_pd


,🔍,displayName,symbol,website,exchange,country,fiveYearAvgDividendYield,dividendRate,dividendYield,currentDividendYield,...,currency,ask,askSize,previousClose,market,marketCap,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,boardRisk
0,Walmart Inc.,Walmart,WMT,https://corporate.walmart.com,NYQ,United States,1.37,0.94,0.90,0.90,...,USD,104.900,54,103.42,us_market,831329140736,99.0834,106.110,77.490,7.0
1,Alphabet Inc.,Alphabet,GOOG,https://abc.xyz,NMS,United States,NaN,0.84,0.34,0.34,...,USD,259.250,1,251.42,us_market,3019553636352,206.4070,253.230,142.660,NaN
2,First Majestic Silver Corp.,First Majestic Silver,AG,https://www.firstmajestic.com,NYQ,Canada,NaN,0.02,0.20,0.20,...,USD,10.190,213,10.37,us_market,4921141760,8.9316,10.900,5.090,3.0
3,Banco Bradesco S.A.,Banco Bradesco,BBD,https://banco.bradesco,NYQ,Brazil,4.49,0.17,5.19,5.14,...,USD,0.000,1792,3.21,us_market,31410081792,2.9574,3.340,1.840,NaN
4,Nokia Oyj,Nokia,NOK,https://www.nokia.com,NYQ,Finland,3.50,0.16,3.32,3.40,...,USD,0.000,970,4.61,us_market,25281744896,4.4138,5.480,3.910,NaN
5,Nordea Bank Abp,None,NDA-DK.CO,https://www.nordea.com,CPH,Finland,6.70,7.01,6.85,6.84,...,DKK,102.650,0,102.40,dk_market,353928839168,97.6408,103.800,72.040,1.0
6,Nordea Bank Abp,None,NDA-FI.HE,https://www.nordea.com,HEL,Finland,7.79,0.94,6.84,6.84,...,EUR,13.755,0,13.74,fi_market,47423463424,13.0926,13.905,9.656,1.0


In [95]:
0.17/3.28


0.05182926829268293